<a href="https://colab.research.google.com/github/BrunoLad/tcc-ufabc/blob/main/tcc_ufabc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Monta o google drive para leitura de dados

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Monta CSV com a lista de todos os livros presente na base, separando-o em um par de nome e título.

In [3]:
import csv
import os

fileList = os.listdir('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/txt')
authorBookList = map(lambda file: file.split('___', 1), fileList)
with open(os.getcwd() + '/books.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Autor', 'Título'])
    for pair in authorBookList:
        writer.writerow(pair)

KeyboardInterrupt: ignored

## Instalação Biblioteca Gutenberg da Pypi

In [2]:
!sudo apt-get install libdb++-dev
!export BERKELEYDB_DIR=/usr
!pip3 install bsddb3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libdb5.3++ libdb5.3++-dev libdb5.3-dev
Suggested packages:
  db5.3-doc
The following NEW packages will be installed:
  libdb++-dev libdb5.3++ libdb5.3++-dev libdb5.3-dev
0 upgraded, 4 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,245 kB of archives.
After this operation, 8,395 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdb5.3++ amd64 5.3.28-13.1ubuntu1.1 [703 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdb5.3-dev amd64 5.3.28-13.1ubuntu1.1 [762 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdb5.3++-dev amd64 5.3.28-13.1ubuntu1.1 [778 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/main amd64 libdb++-dev amd64 1:5.3.21~exp1ubuntu2 [2,340 B]
Fetched 2,245 kB in 1s (2,318 kB/s)
debconf: unable to initiali

In [3]:
!pip install gutenberg

     |████████████████████████████████| 348kB 7.5MB/s 
     |████████████████████████████████| 174kB 43.0MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
  Created wheel for gutenberg: filename=Gutenberg-0.8.1-cp37-none-any.whl size=22237 sha256=83552053bf59f946bd085d197232de675798fcefb1109e95ecaa9f71bbafd372
  Stored in directory: /root/.cache/pip/wheels/90/c2/2e/11c5f40ec9379aa7ac6c143f817a5a784838082f26e14d5dd6
Successfully built gutenberg


## Realiza o filtro dos livros que são de ficção

In [11]:

import os
import re
import json

class BookInfo:
  bookid = None

  def __init__(self, file_name):
    pattern = r'(.+)(?:___)(.+)\.txt'
    book_regex = re.compile(pattern)
    self.filename = file_name
    self.author = book_regex.match(file_name).group(1)
    self.title = book_regex.match(file_name).group(2)


gutenberg_metadata_file = open('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/gutenberg-metadata.json')
gutenberg_metadata = json.load(gutenberg_metadata_file)

# Necessário tratar os casos em que o nome do autor encontra-se na
# linha posterior ao título da obra
def extract_line_from_file(file_name, book):
  with open(file_name, 'r') as read_obj:
    for line in read_obj:
      if book.title in line and book.author in line:
        return line
  return ''

pattern = r'[^_]*$'
digitPattern = r'\d+'
re_bookId = re.compile(digitPattern)
regex = re.compile(pattern, re.MULTILINE)
fileList = os.listdir('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/txt')
newBooks = map(lambda book: BookInfo(book), fileList)
newBooksList = list(newBooks)
print(len(newBooksList))

for book in newBooksList:
  line = extract_line_from_file('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/GUTINDEX.ALL06072021.txt', book)
  match = re_bookId.search(line)
  if match:
    book.bookid = match.group()

filteredBooks = list(filter(lambda book: book.bookid is not None, newBooksList))
fiction_books = filter(lambda book: book.bookid in gutenberg_metadata and re.search(r'[fF]iction', json.dumps(gutenberg_metadata[book.bookid]['subject'])), filteredBooks)
fiction_books_list = list(fiction_books)
print(len(fiction_books_list))

for book in fiction_books_list:
  print(book.title + ' ' +  book.author)

3036
894
The Eyes Have It Philip Kindred Dick
Beyond Lies the Wub Philip Kindred Dick
Second Variety Philip Kindred Dick
The Crystal Crypt Philip Kindred Dick
Bardelys the Magnificent Rafael Sabatini
Captain Blood Rafael Sabatini
Mistress Wilding Rafael Sabatini
Under the Waves R M Ballantyne
Love-at-Arms Rafael Sabatini
St. Martin's Summer Rafael Sabatini
Scaramouche Rafael Sabatini
The Snare Rafael Sabatini
The Trampling of the Lilies Rafael Sabatini
The Historical Nights' Entertainment, First Series Rafael Sabatini
The Tavern Knight Rafael Sabatini
The Lion's Skin Rafael Sabatini
David Balfour, Second Part Robert Louis Stevenson
Catriona Robert Louis Stevenson
Island Nights' Entertainments Robert Louis Stevenson
Fables Robert Louis Stevenson
Kidnapped Robert Louis Stevenson
New Arabian Nights Robert Louis Stevenson
The Merry Men Robert Louis Stevenson
The Strange Case Of Dr. Jekyll And Mr. Hyde Robert Louis Stevenson
The Dynamiter Robert Louis Stevenson
The Black Arrow Robert Louis 